# Forecasting Freshwater Algal Bloom Levels Using Multisource Climate and Water-Quality Data

Course project of **STATS 402: Interdisciplinary Data Analysis**.

**Name:** Ziyue Yin

**NetID:** zy166

## NASA OceanColor Inland Waters (ILW)

### Download

In [2]:
# downloading scripts are independet.

After downloading the datasets, the structure should be shown as follows:

```
datasets/
 ├── ILW/
 │    ├── S3B/2024/CONUS_MO/
 │    │      ├── S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ├── S3B_OLCI_EFRNT.20240201_20240229.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc
 │    ├── Merged/2024/CONUS_DAY/
 │    │      ├── S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ├── S3M_OLCI_EFRNT.20240102.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc
 │    │      ...
```

### Data Structure

First of all, let's glance at the monthly dataset.

In [5]:
import xarray as xr

p = "/dkucc/home/zy166/HAB-forcasting/datasets/ILW/S3B/2024/CONUS_MO/S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc"

ds = xr.open_dataset(p, engine="netcdf4", chunks="auto")
print(ds.dims)
print(list(ds.data_vars))

FrozenMappingWarningOnValuesAccess({'y': 15138, 'x': 26328, 'rgb': 3, 'eightbitcolor': 256})
['rhos_400', 'rhos_412', 'rhos_443', 'rhos_490', 'rhos_510', 'rhos_560', 'rhos_620', 'rhos_665', 'rhos_674', 'rhos_681', 'rhos_709', 'rhos_754', 'rhos_865', 'rhos_884', 'CI_cyano', 'palette']


And also, the daily dataset.

In [4]:
import xarray as xr

p = "/dkucc/home/zy166/HAB-forcasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc"

ds = xr.open_dataset(p, engine="netcdf4", chunks="auto")
print(ds.dims)
print(list(ds.data_vars))

FrozenMappingWarningOnValuesAccess({'y': 15138, 'x': 26328, 'rgb': 3, 'eightbitcolor': 256})
['rhos_400', 'rhos_412', 'rhos_443', 'rhos_490', 'rhos_510', 'rhos_560', 'rhos_620', 'rhos_665', 'rhos_674', 'rhos_681', 'rhos_709', 'rhos_754', 'rhos_865', 'rhos_884', 'CI_cyano', 'palette']


### Target Feature Extraction

#### Utilities

In [18]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import rioxarray

Time Extraction: coords -> attrs -> file names

In [19]:

def infer_time_label(nc_path, ds, product="monthly"):
    """
    返回一个 pandas.Timestamp，尽量从 ds 或文件名推断。
    product: 'monthly' or 'daily'
    """
    # 1) 直接有 time 坐标/变量
    for k in ("time",):
        if k in ds.coords or k in ds.variables:
            try:
                return pd.to_datetime(ds[k].values).to_pydatetime()
            except Exception:
                pass

    # 2) 全局属性（常见于 L3m）
    start = ds.attrs.get("time_coverage_start") or ds.attrs.get("start_time")
    end   = ds.attrs.get("time_coverage_end")   or ds.attrs.get("end_time")
    if start and end:
        try:
            ts = pd.to_datetime(start)
            te = pd.to_datetime(end)
            # 月度：常用“月末”或“中点”；这里给你“中点”更通用
            if product == "monthly":
                return ts + (te - ts) / 2
            else:
                # 日产品：用开始时间
                return ts
        except Exception:
            pass

    # 3) 从文件名解析
    fn = nc_path.split("/")[-1]
    if product == "monthly":
        # ...YYYYMMDD_YYYYMMDD.L3m.MO...
        m = re.search(r"\.(\d{8})_(\d{8})\.L3m\.MO\.", fn)
        if m:
            b, e = m.group(1), m.group(2)
            ts = pd.to_datetime(b, format="%Y%m%d")
            te = pd.to_datetime(e, format="%Y%m%d")
            return ts + (te - ts) / 2
    else:
        # ...YYYYMMDD.L3m.DAY...
        m = re.search(r"\.(\d{8})\.L3m\.DAY\.", fn)
        if m:
            return pd.to_datetime(m.group(1), format="%Y%m%d")

    # 实在没有，就抛异常，提示检查文件
    raise ValueError("Cannot infer time from dataset or filename: " + fn)

Quality Prune

In [20]:
def clean_ci(da: xr.DataArray) -> xr.DataArray:
    """
    依据变量属性做物理范围过滤，并去掉接近下界的“近零”点。
    """
    vmin = float(da.attrs.get("valid_min", np.nan))
    vmax = float(da.attrs.get("valid_max", np.nan))
    if np.isfinite(vmin):
        da = da.where(da >= vmin)
    if np.isfinite(vmax):
        da = da.where(da <= vmax)

    # 去掉接近下界的小值（阈值可按需要调整）
    thr = max(vmin, 5e-5) if np.isfinite(vmin) else 5e-5
    da = da.where(da > thr)

    return da

For a single .nc file, get all the lakes

In [21]:
def extract_lakes_from_nc(nc_path: str,
                          lakes_gdf: gpd.GeoDataFrame,
                          lake_id_col: str,
                          product: str) -> pd.DataFrame:
    """
    nc_path: 单个 NetCDF 文件（S3B monthly 或 S3M daily）
    lakes_gdf: 包含 lake_id 和 geometry 的 GeoDataFrame（EPSG:4326）
    product: 'monthly' | 'daily'
    返回：每个湖一行（该文件的时间戳）
    """
    ds = xr.open_dataset(nc_path, engine="netcdf4", chunks="auto")
    t  = infer_time_label(nc_path, ds, product=product)

    da = ds["CI_cyano"]
    da = set_spatial_dims_safe(da)
    da = clean_ci(da)

    rows = []
    for _, row in lakes_gdf.iterrows():
        lid  = row[lake_id_col]
        geom = [row.geometry]  # rioxarray.clip 需要 list

        try:
            clipped = da.rio.clip(geom, lakes_gdf.crs, drop=True)
            valid   = clipped.where(np.isfinite(clipped))
            n_valid = int(valid.count().compute().values)
            if n_valid == 0:
                mean_val = np.nan
                p90      = np.nan
            else:
                mean_val = float(valid.mean().compute().values)
                p90      = float(valid.quantile(0.9).compute().values)
        except Exception:
            mean_val, p90, n_valid = np.nan, np.nan, 0

        rows.append({
            "lake_id": lid,
            "time":   pd.to_datetime(t),
            "product": product,
            "CI_mean": mean_val,
            "CI_p90":  p90,
            "n_valid": n_valid,
            "src":     Path(nc_path).name,
        })

    ds.close()
    return pd.DataFrame(rows)

Process monthly data in batches

In [22]:
def run_monthly(monthly_dir: str,
                lakes_fp: str,
                lake_id_col: str,
                out_parquet: str):
    """
    monthly_dir: 目录内文件形如 S3B_OLCI_EFRNT.*.L3m.MO.ILW_CONUS...nc
    lakes_fp:    湖泊边界（gpkg/shp，需 EPSG:4326）
    """
    gdf = gpd.read_file(lakes_fp)
    if gdf.crs is None:
        raise ValueError("湖泊文件缺少 CRS，请确保为 EPSG:4326")
    gdf = gdf.to_crs(4326)[[lake_id_col, "geometry"]].dropna()

    out_rows = []
    for fp in sorted(Path(monthly_dir).glob("S3B_OLCI_EFRNT.*.L3m.MO.*.nc")):
        df_one = extract_lakes_from_nc(str(fp), gdf, lake_id_col, product="monthly")
        out_rows.append(df_one)

    if not out_rows:
        print("No monthly files found.")
        return

    df_all = pd.concat(out_rows, ignore_index=True)
    Path(out_parquet).parent.mkdir(parents=True, exist_ok=True)
    df_all.to_parquet(out_parquet, index=False)
    print(f"[monthly] saved → {out_parquet}  ({len(df_all)} rows)")

Process daily data in batches

In [23]:
def run_daily(daily_dir: str,
              lakes_fp: str,
              lake_id_col: str,
              out_parquet: str):
    """
    daily_dir: 目录内文件形如 S3M_OLCI_EFRNT.*.L3m.DAY.ILW_CONUS...nc
    """
    gdf = gpd.read_file(lakes_fp)
    if gdf.crs is None:
        raise ValueError("湖泊文件缺少 CRS，请确保为 EPSG:4326")
    gdf = gdf.to_crs(4326)[[lake_id_col, "geometry"]].dropna()

    out_rows = []
    for fp in sorted(Path(daily_dir).glob("S3M_OLCI_EFRNT.*.L3m.DAY.*.nc")):
        df_one = extract_lakes_from_nc(str(fp), gdf, lake_id_col, product="daily")
        out_rows.append(df_one)

    if not out_rows:
        print("No daily files found.")
        return

    df_all = pd.concat(out_rows, ignore_index=True)
    Path(out_parquet).parent.mkdir(parents=True, exist_ok=True)
    df_all.to_parquet(out_parquet, index=False)
    print(f"[daily] saved → {out_parquet}  ({len(df_all)} rows)")

Spatial Coordination

In [24]:
def set_spatial_dims_safe(da: xr.DataArray) -> xr.DataArray:
    """
    尝试为 L3m 网格设置 rioxarray 所需的空间维和 CRS。
    先使用 x/y；失败则尝试 lon/lat；再不行退化为最后两个维度命名为 x/y。
    """
    if "x" in da.dims and "y" in da.dims:
        out = da.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
        return out

    if "lon" in da.dims and "lat" in da.dims:
        out = da.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)
        return out

    # 退化方案：尝试把最后两个维度当作 x/y
    if len(da.dims) >= 2:
        dims = list(da.dims)
        ydim, xdim = dims[-2], dims[-1]
        out = da.rename({xdim: "x", ydim: "y"})
        out = out.rio.write_crs(4326)
        out = out.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
        return out

    raise ValueError("Cannot determine spatial dims for CI_cyano")

#### Scale 1: In general

##### Monthly

Here, we use the **S3B Monthly** data. One month per row.

In [9]:
import glob, numpy as np, pandas as pd, xarray as xr
from pathlib import Path

monthly_dir = Path("/dkucc/home/zy166/HAB-forcasting/datasets/ILW/S3B/2024/CONUS_MO")
out_csv = monthly_dir/"ci_cyano_monthly_mean.csv"

rows = []
for fp in sorted(monthly_dir.glob("S3B_OLCI_EFRNT.*.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc")):
    ds = xr.open_dataset(fp, engine="netcdf4", chunks="auto")
    da = ds["CI_cyano"]

    vmin = float(da.attrs.get("valid_min", np.nan))
    vmax = float(da.attrs.get("valid_max", np.nan))
    if np.isfinite(vmin): da = da.where(da >= vmin)
    if np.isfinite(vmax): da = da.where(da <= vmax)

    m   = float(da.where(np.isfinite(da)).mean().compute().values)
    p90 = float(da.where(np.isfinite(da)).quantile(0.9).compute().values)
    t   = infer_time_label(str(fp), ds, product="monthly")

    rows.append({"time": pd.to_datetime(t), "CI_mean": m, "CI_p90": p90,
                 "n_valid": int(da.count().compute().values)})
    ds.close()

df_mo = pd.DataFrame(rows).sort_values("time").reset_index(drop=True)
df_mo.to_csv(out_csv, index=False)
df_mo.head()

,time,CI_mean,CI_p90,n_valid
0,2024-01-16 16:53:28.500000+00:00,0.000160,0.00005,9965844
1,2024-02-15 17:12:11.500000+00:00,0.000157,0.00005,10102417
2,2024-03-16 16:44:09+00:00,0.000135,0.00005,9452529
3,2024-04-16 04:52:02+00:00,0.000141,0.00005,10500573
4,2024-05-16 17:01:29+00:00,0.000149,0.00005,11822670


##### Daily

Here, we use the **S3M Daily** data. One day per row.

In [10]:
daily_dir = Path("/dkucc/home/zy166/HAB-forcasting/datasets/ILW/Merged/2024/CONUS_DAY")
out_csv = daily_dir/"ci_cyano_daily_mean.csv"

rows = []
for fp in sorted(daily_dir.glob("S3M_OLCI_EFRNT.*.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc")):
    ds = xr.open_dataset(fp, engine="netcdf4", chunks="auto")
    da = ds["CI_cyano"]

    vmin = float(da.attrs.get("valid_min", np.nan))
    vmax = float(da.attrs.get("valid_max", np.nan))
    if np.isfinite(vmin): da = da.where(da >= vmin)
    if np.isfinite(vmax): da = da.where(da <= vmax)

    m   = float(da.where(np.isfinite(da)).mean().compute().values)
    p90 = float(da.where(np.isfinite(da)).quantile(0.9).compute().values)
    t   = infer_time_label(str(fp), ds, product="daily")

    rows.append({"date": pd.to_datetime(t), "CI_mean": m, "CI_p90": p90,
                 "n_valid": int(da.count().compute().values)})
    ds.close()

df_day = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)
df_day.to_csv(out_csv, index=False)
df_day.head()

,date,CI_mean,CI_p90,n_valid
0,2024-01-01 13:52:31+00:00,0.000127,0.00005,5055519
1,2024-01-02 13:36:49+00:00,0.000094,0.00005,5360242
2,2024-01-03 14:02:06+00:00,0.000112,0.00005,4457855
3,2024-01-04 13:40:26+00:00,0.000087,0.00005,5619796
4,2024-01-05 13:48:41+00:00,0.000096,0.00005,5554313


In [11]:
import xarray as xr, numpy as np, pandas as pd, glob, re

# 任选一个月度/日度文件
f_mo = sorted(glob.glob("/dkucc/home/zy166/HAB-forcasting/datasets/ILW/S3B/2024/CONUS_MO/S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc"))[0]
f_da = sorted(glob.glob("/dkucc/home/zy166/HAB-forcasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc"))[0]

for f in [f_mo, f_da]:
    ds = xr.open_dataset(f, engine="netcdf4", chunks="auto")
    da = ds["CI_cyano"]
    print("\n==", f)
    print("dims:", da.dims, da.sizes)
    print("attrs:", {k: da.attrs.get(k) for k in ["valid_min","valid_max","_FillValue","scale_factor","add_offset"]})
    # 统计分布（不裁剪，看看原始情况）
    s = da.load().values.flatten()
    s = s[np.isfinite(s)]
    q = np.quantile(s, [0.0, 0.1, 0.5, 0.9, 1.0])
    print("raw quantiles:", q)
    ds.close()


== /dkucc/home/zy166/HAB-forcasting/datasets/ILW/S3B/2024/CONUS_MO/S3B_OLCI_EFRNT.20240101_20240131.L3m.MO.ILW_CONUS.V5.all.CONUS.300m.nc
dims: ('y', 'x') Frozen({'y': 15138, 'x': 26328})
attrs: {'valid_min': np.float32(5e-05), 'valid_max': np.float32(0.05), '_FillValue': None, 'scale_factor': None, 'add_offset': None}
raw quantiles: [4.99999805e-05 4.99999951e-05 4.99999987e-05 5.00000024e-05
 7.23447204e-02]

== /dkucc/home/zy166/HAB-forcasting/datasets/ILW/Merged/2024/CONUS_DAY/S3M_OLCI_EFRNT.20240101.L3m.DAY.ILW_CONUS.V5.all.CONUS.300m.nc
dims: ('y', 'x') Frozen({'y': 15138, 'x': 26328})
attrs: {'valid_min': np.float32(5e-05), 'valid_max': np.float32(0.05), '_FillValue': None, 'scale_factor': None, 'add_offset': None}
raw quantiles: [4.99999915e-05 4.99999951e-05 4.99999987e-05 5.00000024e-05
 8.10017735e-02]


#### Scale 2: Specific Lake(s)/Area(s)

In [25]:
# Monthly
run_monthly(
    monthly_dir="/dkucc/home/zy166/HAB-forcasting/datasets/ILW/S3B/2024/CONUS_MO",
    lakes_fp="/dkucc/home/zy166/HAB-forcasting/shapes/lakes_conus.gpkg",
    lake_id_col="lake_id",
    out_parquet="/dkucc/home/zy166/HAB-forcasting/data/processed/lake_ci_monthly.parquet"
)

DataSourceError: /dkucc/home/zy166/HAB-forcasting/shapes/lakes_conus.gpkg: No such file or directory

In [ ]:
# Daily
run_daily(
    daily_dir="/dkucc/home/zy166/HAB-forcasting/datasets/ILW/Merged/2024/CONUS_DAY",
    lakes_fp="/dkucc/home/zy166/HAB-forcasting/shapes/lakes_conus.gpkg",
    lake_id_col="lake_id",
    out_parquet="/dkucc/home/zy166/HAB-forcasting/data/processed/lake_ci_daily.parquet"
)

### Time Alignment & Missing Value Handling

### Standard Preprocessed Format